##1. 데이터 읽기

In [ ]:
import pandas as pd
df_analysis = pd.read_csv("/content/combined_trips_reuploaded.csv", encoding='cp949')

columns = df_analysis.columns
columns

df_analysis_corrected = pd.read_csv("/content/combined_trips_reuploaded.csv", encoding='cp949', sep=';')

corrected_columns = df_analysis_corrected.columns
corrected_columns

##2. 데이터 정제 및 분석




- 중복된 컬럼을 제거하였습니다.

In [ ]:
duplicate_columns = df_analysis_corrected.columns[df_analysis_corrected.columns.duplicated()].tolist()

identical_columns_dict = {}

for column in duplicate_columns:
    duplicate_mask = df_analysis_corrected.columns == column

    if df_analysis_corrected.loc[:, duplicate_mask].nunique(dropna=False).iloc[0] == 1:
        identical_columns_dict[column] = df_analysis_corrected.columns[duplicate_mask].tolist()

for columns in identical_columns_dict.values():
    df_analysis_corrected = df_analysis_corrected.drop(columns=columns[1:])

remaining_columns = df_analysis_corrected.columns
remaining_columns


- 위험도를 설정하기 위해 도메인 지식을 통해서 관련 있는 5가지 변수를 추출 하였습니다. 그리고 이 변수들을 통해서 위험도 점수를 측정하였습니다

1) 배터리 온도:

0 ~ 25: 점수 1 (낮은 위험)

26 ~ 35: 점수 2 (보통 위험)

36이상: 점수 3 (높은 위험)

2) 배터리 전압:

0~3.2V: 점수 1 (낮은 위험)

3.3~3.7V: 점수 2 (보통 위험)

3.8V 이상: 점수 3 (높은 위험)

3) 배터리 전류:

0~100A: 점수 1 (낮은 위험)

101~200A: 점수 2 (보통 위험)

201A 이상: 점수 3 (높은 위험)

4) SoC (State of Charge):

0~20%: 점수 3 (높은 위험)

21~80%: 점수 1 (낮은 위험)

81~100%: 점수 2 (보통 위험)


5) 주변 환경 온도:
0~25도: 점수 1 (낮은 위험)

26~35도: 점수 2 (보통 위험)

36도 이상: 점수 3 (높은 위험)




- 리스크에 관한 Feature를 제작하였습니다.(해당 Feature는  각 요소별 위험 점수를 합산한 값)


- 이후, 총 점수를 기반으로 '낮음', '중간', '높음'의 세 가지 카테고리로 위험도를 분류하겠습니다.

In [ ]:
import numpy as np

df_analysis_corrected['Temp_Risk'] = np.select(
    [df_analysis_corrected['Battery Temperature [캜]'] <= 25,
     df_analysis_corrected['Battery Temperature [캜]'].between(26, 35),
     df_analysis_corrected['Battery Temperature [캜]'] >= 36],
    [1, 2, 3]
)

df_analysis_corrected['Voltage_Risk'] = np.select(
    [df_analysis_corrected['Battery Voltage [V]'] <= 3.2,
     df_analysis_corrected['Battery Voltage [V]'].between(3.3, 3.7),
     df_analysis_corrected['Battery Voltage [V]'] >= 3.8],
    [1, 2, 3]
)

df_analysis_corrected['Current_Risk'] = np.select(
    [df_analysis_corrected['Battery Current [A]'] <= 100,
     df_analysis_corrected['Battery Current [A]'].between(101, 200),
     df_analysis_corrected['Battery Current [A]'] >= 201],
    [1, 2, 3]
)

df_analysis_corrected['SoC_Risk'] = np.select(
    [df_analysis_corrected['SoC [%]'] <= 20,
     df_analysis_corrected['SoC [%]'].between(21, 80),
     df_analysis_corrected['SoC [%]'] >= 81],
    [3, 1, 2]
)

df_analysis_corrected['Ambient_Temp_Risk'] = np.select(
    [df_analysis_corrected['Ambient Temperature [캜]'] <= 25,
     df_analysis_corrected['Ambient Temperature [캜]'].between(26, 35),
     df_analysis_corrected['Ambient Temperature [캜]'] >= 36],
    [1, 2, 3]
)

df_analysis_corrected['Total_Risk'] = df_analysis_corrected['Temp_Risk'] + df_analysis_corrected['Voltage_Risk'] + df_analysis_corrected['Current_Risk'] + df_analysis_corrected['SoC_Risk'] + df_analysis_corrected['Ambient_Temp_Risk']


df_analysis_corrected['Risk_Category'] = pd.cut(df_analysis_corrected['Total_Risk'], bins=[5, 7, 9, 15], labels=['Low', 'Medium', 'High'], right=False)

df_analysis_corrected[['Risk_Category', 'Total_Risk']].head()

여기에 차량에서 사용되는 전력 또한 배터리 사용량을 높이고 이는 배터리 과열로 인한 위험도를 높인다고 판단하였습니다


그래서 데이터 셋 안에 있는 전력 생산과 관련된 데이터 분석을 진행하였습니다


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 6))
df_analysis_corrected.plot(x='Time [s]', y=['Heating Power CAN [kW]', 'AirCon Power [kW]'], figsize=(14, 6))
plt.title('Trend of Heating Power and Air Conditioning Power over Time')
plt.ylabel('Power [kW]')
plt.grid(True)
plt.show()

"Heating Power CAN [kW]"는 전체 시간 동안 일정하게 낮은 값을 유지하며, 특별한 패턴이나 급격한 변화가 없습니다.


"AirCon Power [kW]"는 일부 시간 동안 급격한 증가와 감소를 보이며, 주기적인 패턴이 관찰됩니다.


이를 통해 난방 전력은 일정하게 사용되는 반면, 에어콘의 전력 사용량은 시간에 따라 변동이 크다는 것을 알 수 있습니다.

저희는 난방 전력 생산량과 에어컨 전력 사용량을 통해 두 전력 생산값이라는 새로운 Feature를 제작하였습니다

In [ ]:
df_analysis_corrected['Total Vehicle Power Usage [kW]'] = df_analysis_corrected['Heating Power CAN [kW]'] + df_analysis_corrected['AirCon Power [kW]']

df_analysis_corrected[['Time [s]', 'Heating Power CAN [kW]', 'AirCon Power [kW]', 'Total Vehicle Power Usage [kW]']].head()

이 새로운 Feature를 이용하여 새로운 변수를 위험도에 추가하였습니다.

In [ ]:

df_analysis_corrected['Vehicle_Power_Risk'] = np.select(
    [df_analysis_corrected['Total Vehicle Power Usage [kW]'] <= 2,
     df_analysis_corrected['Total Vehicle Power Usage [kW]'].between(2, 4),
     df_analysis_corrected['Total Vehicle Power Usage [kW]'] >= 4],
    [1, 2, 3]
)


df_analysis_corrected['Total_Risk'] = df_analysis_corrected['Temp_Risk'] + df_analysis_corrected['Voltage_Risk'] + df_analysis_corrected['Current_Risk'] + df_analysis_corrected['SoC_Risk'] + df_analysis_corrected['Ambient_Temp_Risk']

df_analysis_corrected['Risk_Category'] = pd.cut(df_analysis_corrected['Total_Risk'], bins=[5, 7, 9, 15], labels=['Low', 'Medium', 'High'], right=False)

df_analysis_corrected[['Risk_Category', 'Total_Risk']].head()


df_analysis_corrected['Total_Risk'] = df_analysis_corrected['Total_Risk'] + df_analysis_corrected['Vehicle_Power_Risk']

df_analysis_corrected['Risk_Category'] = pd.cut(df_analysis_corrected['Total_Risk'], bins=[6, 8, 10, 17], labels=['Low', 'Medium', 'High'], right=False)

df_analysis_corrected[['Risk_Category', 'Total_Risk']].head()

해당 과정이후 위험도 컬럼에 대한 가중치를 설정하여 가중합을 사용해 "Total_Risk"를 계산하였습니다.

그리고 가중치는 통계적 방법을 이용하여 데이터를 바탕으로 각 위험도 컬럼의 중요도를 계산하였습니다.

우선 랜덤 포레스트를 사용하여 각 위험도의 컬럼의 중요도를 확인해보았습니다.

"Total_Risk"를 종속 변수로, 나머지 위험도 컬럼들(Temp_Risk, Voltage_Risk, Current_Risk, SoC_Risk, Ambient_Temp_Risk, Vehicle_Power_Risk)을 독립 변수로 사용하여 랜덤 포레스트 모델을 학습하겠습니다. 이후, 각 변수의 중요도를 계산하겠습니다.


In [ ]:
from sklearn.ensemble import RandomForestRegressor

X = df_analysis_corrected[['Temp_Risk', 'Voltage_Risk', 'Current_Risk', 'SoC_Risk', 'Ambient_Temp_Risk', 'Vehicle_Power_Risk']]
y = df_analysis_corrected['Total_Risk']

rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(X, y)

feature_importances = rf.feature_importances_

features_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': feature_importances
}).sort_values(by='Importance', ascending=False)
features_df

위의 값을 확인해본 결과 배터리 온도 위험도와 전압의 위험도가 없는 것으로 나왔습니다. 하지만 배터리 온도와 위험도는 큰 상관관계를 가지고 있다고 생각하여 상관관계를 구해보았습니다.

In [ ]:

battery_temp_risk_correlation = df_analysis_corrected['Battery Temperature [캜]'].corr(df_analysis_corrected['Total_Risk'])
battery_temp_risk_correlation

상관계수를 구해본 결과 위와 같이 배터리 온도와 배터리 위험도는 높은 상관 관계를 가진다는 것을 알게 되었습니다.

In [ ]:

risk_columns = ['Temp_Risk', 'Voltage_Risk', 'Current_Risk', 'SoC_Risk', 'Ambient_Temp_Risk', 'Vehicle_Power_Risk', 'Total_Risk']
correlation_matrix = df_analysis_corrected[risk_columns].corr()


total_risk_correlation = correlation_matrix['Total_Risk'].drop('Total_Risk')
total_risk_correlation

이에 대해서 이전의 랜덤 포레스트 중요도 분석에서 "Temp_Risk"의 중요도가 0으로 나온 것의 원인을 고민해 보았고

다중공선성으 다른 변수들과 "Temp_Risk" 사이에 강한 상관 관계가 있을 경우, 랜덤 포레스트는 중복된 정보를 갖는 변수의 중요도를 낮게 평가할 수 있다고 생각되고 아니면,데이터 분포에서 만약 "Temp_Risk"의 값이 데이터셋에서 큰 변동을 보이지 않는다면, 이 변수는 목표 변수인 "Total_Risk"를 예측하는 데 큰 영향을 주지 않았을 수도 있다고 생각하였습니다.

그 밖에 모델의 한계로 인해 랜덤 포레스트는 데이터의 특성과 분포에 따라 각 변수의 중요도를 다르게 평가되었다는 가정을 하였습니다.

하지만 상관 분석 결과를 통해 배터리 온도와 위험도 간에는 강한 상관 관계가 있다는 것을 확인했고 따라서 배터리 온도는 위험도 예측에 중요한 변수로 간주하였습니다.


그렇기 때문에 위와 같이 다른 중요한 변수들에 대해서도 상관관계를 살펴보았습니다.

그래서 아래와 같이 배터리 위험도에 대한 수식을 상관관계를 이용하여 작성해보았습니다.

In [ ]:

df_analysis_corrected['Total_Risk_new'] = (total_risk_correlation['Temp_Risk'] * df_analysis_corrected['Temp_Risk'] +
                                           total_risk_correlation['Current_Risk'] * df_analysis_corrected['Current_Risk'] +
                                           total_risk_correlation['SoC_Risk'] * df_analysis_corrected['SoC_Risk'] +
                                           total_risk_correlation['Ambient_Temp_Risk'] * df_analysis_corrected['Ambient_Temp_Risk'] +
                                           total_risk_correlation['Vehicle_Power_Risk'] * df_analysis_corrected['Vehicle_Power_Risk'])

df_analysis_corrected[['Temp_Risk', 'Current_Risk', 'SoC_Risk', 'Ambient_Temp_Risk', 'Vehicle_Power_Risk', 'Total_Risk', 'Total_Risk_new']].head()

최종적으로 위의 파일을 battery.csv라는 파일로 저장하였습니다.

In [ ]:
df_analysis_corrected.to_csv("battery.csv", index = False)

##3 데이터 전처리


결측치를 확인해보았습니다.

In [ ]:
import pandas as pd

battery_data = pd.read_csv('./battery.csv')

battery_data.head()

missing_values = battery_data.isnull().sum()

missing_values[missing_values > 0]

결측치가 존재했고 결측치의 동향을 한번 확인해보았습니다.

In [ ]:
missing_columns = missing_values[missing_values > 0].index
battery_data[battery_data[missing_columns[0]].isnull()][missing_columns].head()

해당 결측치는 삭제는 행전체가 NaN이고 갯수가 많지 않아서 다른 값들로 대체하는 것이 아닌 삭제를 택하였습니다.

In [ ]:
battery_data_cleaned = battery_data.dropna(subset=missing_columns)

remaining_missing = battery_data_cleaned.isnull().sum()
remaining_missing_values = remaining_missing[remaining_missing > 0]

remaining_missing_values

##4 데이터 모델 생성

이제 Total_Risk를 이용하여 모델 제작을 진행하였습니다.


결측치를 제거하여 데이터 훈련 세트와 테스트 세트를 분할하였습니다.

In [ ]:
from sklearn.model_selection import train_test_split

battery_data_no_missing_cols = battery_data.drop(columns=missing_columns)

X = battery_data_no_missing_cols.drop('Total_Risk', axis=1)
y = battery_data_no_missing_cols['Total_Risk']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


특정 열에서 문자열 값을 실수로 변환 할 수 없어 time에서는 ,를 제거하고 숫자로 변환하고 risk_category는 범주형이라서 숫자로 인코딩하였습니다. 그리고 모델을 랜덤포레스트 알고리즘으로 학습시키고 오차를 확인해보았습니다.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

X_train['Time [s]'] = X_train['Time [s]'].str.replace(',', '').astype(float)
X_test['Time [s]'] = X_test['Time [s]'].str.replace(',', '').astype(float)

risk_category_mapping = {'Low': 1, 'Medium': 2, 'High': 3}
X_train['Risk_Category'] = X_train['Risk_Category'].map(risk_category_mapping)
X_test['Risk_Category'] = X_test['Risk_Category'].map(risk_category_mapping)

rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)

mse

전처리 후 MSE는 4.97×10 −5 였습니다. 이번에는 선형회귀를 이용하여 모델을 학습시켜보았습니다.

In [ ]:
from sklearn.linear_model import LinearRegression

lr_model = LinearRegression()

lr_model.fit(X_train, y_train)

y_pred_lr = lr_model.predict(X_test)

mse_lr = mean_squared_error(y_test, y_pred_lr)

mse_lr

오차를 더 줄이기 위해서 연관성이 높은 컬럼들만을 사용하여 오차를 줄여보았습니다.일단 모든 컬럼을 확인하였습니다.

In [ ]:
all_columns = battery_data_no_missing_cols.columns.tolist()
all_columns

이중 관련성이 높다고 판단한 'Time [s]', 'Velocity [km/h]','Battery Voltage [V]', 'Battery Current [A]',
 'Battery Temperature [캜]', 'SoC [%]',  'Ambient Temperature [캜]',  'Temp_Risk',
 'Voltage_Risk', 'Current_Risk','SoC_Risk', 'Ambient_Temp_Risk', 'Total Vehicle Power Usage [kW]', 'Vehicle_Power_Risk' 을 사용해보았습니다.

In [ ]:
selected_columns = [
    'Time [s]', 'Velocity [km/h]', 'Battery Voltage [V]', 'Battery Current [A]',
    'Battery Temperature [캜]', 'SoC [%]', 'Ambient Temperature [캜]', 'Temp_Risk',
    'Voltage_Risk', 'Current_Risk', 'SoC_Risk', 'Ambient_Temp_Risk',
    'Total Vehicle Power Usage [kW]', 'Vehicle_Power_Risk'
]

X_train_selected = X_train[selected_columns]
X_test_selected = X_test[selected_columns]

lr_model_selected = LinearRegression()
lr_model_selected.fit(X_train_selected, y_train)

y_pred_selected = lr_model_selected.predict(X_test_selected)

mse_selected = mean_squared_error(y_test, y_pred_selected)

mse_selected

위와 같이 오차가 줄어들어 결과값은 더 좋아졌습니다. 이번에는 그래디언트 부스팅을 사용해보았습니다.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)

gb_model.fit(X_train_selected, y_train)

y_pred_gb = gb_model.predict(X_test_selected)

mse_gb = mean_squared_error(y_test, y_pred_gb)

mse_gb

오차는 랜덤포레스트보다 적으며 선형회귀보다는 큰 결과 값을 얻었습니다. 이번에는 K-Nearest Neighbors (KNN), Support Vector Machine (SVM) for Regression
,Decision Tree Regressor 예측 모델을 사용해보고 성능을 분석해보았습니다.


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

knn_model = KNeighborsRegressor(n_neighbors=5)
svm_model = SVR(kernel='linear', C=1)
dt_model = DecisionTreeRegressor(random_state=42)

knn_model.fit(X_train_selected, y_train)
svm_model.fit(X_train_selected, y_train)
dt_model.fit(X_train_selected, y_train)

y_pred_knn = knn_model.predict(X_test_selected)
y_pred_svm = svm_model.predict(X_test_selected)
y_pred_dt = dt_model.predict(X_test_selected)

mse_knn = mean_squared_error(y_test, y_pred_knn)
mse_svm = mean_squared_error(y_test, y_pred_svm)
mse_dt = mean_squared_error(y_test, y_pred_dt)

mse_knn, mse_svm, mse_dt

모든 학습 방법으로 확인해본 오차값을 정리해보았습니다.

In [ ]:
mse_summary = {
    'Linear Regression': mse_selected,
    'Random Forest': mse_rf_selected,
    'Gradient Boosting': mse_gb,
    'K-Nearest Neighbors (KNN)': mse_knn,
    'Support Vector Machine (SVM)': mse_svm,
    'Decision Tree': mse_dt
}

mse_summary_df = pd.DataFrame(list(mse_summary.items()), columns=['Model', 'MSE'])
mse_summary_df.sort_values(by="MSE", ascending=True)

이 중, Linear Regression 모델이 가장 낮은 MSE 값을 보이며 가장 좋은 성능을 나타냈었고 저희는 이 모델을 이용하여 배터리 위험도를 예측하는 모델을 완성시켰습니다.